# User Item Biases
* See the corresponding file in `../../TrainingAlphas` for more details

In [ ]:
source = "ExplicitUserItemBiases";

In [ ]:
import NBInclude: @nbinclude
@nbinclude("../Alpha.ipynb");
@nbinclude("../../TrainingAlphas/Explicit/ExplicitUserItemBiasesBase.ipynb");

In [ ]:
function train_model(training, λ, μ, a)
    λ_u, _, λ_wu, λ_wa = λ
    @info "training model with parameters [$λ_u, $λ_wu, $λ_wa]"
    users, items, ratings = training.user, training.item, training.rating
    weights =
        powerdecay(length(training.item), log(λ_wu)) .* powerdecay(
            get_counts("training", false; by_item = true, per_rating = false)[training.item],
            log(λ_wa),
        )
    u = zeros(eltype(λ_u), maximum(users))
    ρ_u = zeros(eltype(u), length(u), Threads.nthreads())
    Ω_u = zeros(eltype(u), length(u), Threads.nthreads())
    update_users!(users, items, ratings, weights, u, a, λ_u, ρ_u, Ω_u; μ = μ)
    u
end;

In [ ]:
function compute_alpha()
    training = get_recommendee_split(false)
    params = read_params(source)
    u = train_model(training, params["λ"], mean(params["u"]), params["a"])
    @info u
    preds = make_prediction(fill(1, num_items()), collect(1:num_items()), u, params["a"])
    write_recommendee_alpha(preds, source)
end;

In [ ]:
compute_alpha();